In [1]:
import gym 
from gym.wrappers import RecordVideo

In [2]:
env_name = 'CarRacing-v0'

In [3]:
env = gym.make(env_name)
env = RecordVideo(env, f'{env_name}-video', episode_trigger=lambda _: True)

/anaconda/envs/azureml_py38/lib/python3.8/site-packages/gym/wrappers/record_video.py:41: UserWarning: WARN: Overwriting existing videos at /mnt/batch/tasks/shared/LS_root/mounts/clusters/gpucomputer/code/Users/fkhan/RL-reserach/CarRacing-v0-video folder (try specifying a different `video_folder` for the `RecordVideo` wrapper if this is not desired)
  logger.warn(


In [4]:
def test_environment():
    episodes = 10
    observation = env.reset()
    for episode in range(episodes): 
        state = env.reset()
        done = False 
        score = 0 
        while not done: 
            action = env.action_space.sample()
            observation, reward, done, info = env.step(action)
            score += reward 
        print(f'Episode={episode+1} Score:{score}')
    env.close()

In [5]:
from pyvirtualdisplay import Display
display = Display(visible=0, size=(1400, 900))
display.start()

In [9]:
test_environment()

Track generation: 1168..1464 -> 296-tiles track
Track generation: 1171..1468 -> 297-tiles track
Episode=1 Score:-32.43243243243294
Track generation: 1028..1295 -> 267-tiles track
Episode=2 Score:-28.57142857142894
Track generation: 1136..1424 -> 288-tiles track
Episode=3 Score:-33.79790940766592
Track generation: 1272..1594 -> 322-tiles track
Episode=4 Score:-37.69470404984479
Track generation: 1168..1464 -> 296-tiles track
Episode=5 Score:-35.59322033898358
Track generation: 1031..1293 -> 262-tiles track
Episode=6 Score:-27.20306513409994
Track generation: 1001..1262 -> 261-tiles track
Episode=7 Score:-23.0769230769233
Track generation: 1056..1324 -> 268-tiles track
Episode=8 Score:-25.093632958801678
Track generation: 1151..1443 -> 292-tiles track
Episode=9 Score:-31.27147766323071
Track generation: 1068..1339 -> 271-tiles track
Episode=10 Score:-25.925925925926244


In [12]:
from stable_baselines3 import DDPG
from stable_baselines3.common.noise import NormalActionNoise, OrnsteinUhlenbeckActionNoise
from stable_baselines3.common.vec_env import DummyVecEnv

In [13]:
import numpy as np 
env = gym.make(env_name)
env = DummyVecEnv([lambda: env])
n_actions = env.action_space.shape[-1]
action_noise = NormalActionNoise(mean=np.zeros(n_actions), sigma=0.1 * np.ones(n_actions))
print(f'{n_actions=}')
model_name = 'ddpg'
model = DDPG("MlpPolicy", env, action_noise=action_noise, verbose=1)

n_actions=3
Using cuda device
Wrapping the env in a VecTransposeImage.


In [14]:
model.learn(total_timesteps=100, log_interval=5)
model.save(f'{env_name}-{model_name}')

Track generation: 1245..1566 -> 321-tiles track
Track generation: 948..1189 -> 241-tiles track


In [15]:
def record_trained_model(model): 
    episodes = 10
    env = gym.make(env_name)
    env = RecordVideo(env, f'{env_name}-{model_name}-video', episode_trigger=lambda _: True)  
    
    for episode in range(episodes): 
        observation = env.reset()
        done = False 
        score = 0 
        while not done: 
            action, _states = model.predict(observation)
            observation, reward, done, info = env.step(action)
            score += reward 
        print(f'Episode={episode+1} Score:{score}')
    env.close()

In [16]:
record_trained_model(model)

ValueError: At least one stride in the given numpy array is negative, and tensors with negative strides are not currently supported. (You can probably work around this by making a copy of your array  with array.copy().) 